Source: YellowKing Team
Modified for use in Bengali.AI Speech Recognition competition

We loaded the data from bengaliAI huggingface and made the following adjustments:

1. Normalized using bnunicodenormalizer
2. Tokenized using Wav2Vec2Processor.from_pretrained("arijitx/wav2vec2-xls-r-300m-bengali")
3. Removed special characters
4. Resampled at 16000Hz (required for wav2vec2 training later on)
5. Trimmed silence with the threshhold (amplitude < max_amplitude/30 ). This makes training faster
6. Uploaded to huggingface: https://huggingface.co/Lancelot53

In [ ]:
# %%capture
# !pip install transformers
# !pip install jiwer
# !apt install git-lfs

In [5]:
from huggingface_hub import notebook_login
from datasets import load_dataset
import pandas as pd

In [7]:
df = pd.read_csv("train_mp3_subsets/train_subset_1/metadata.csv")
df

,file_name,transcription
0,014fe4c1316a.mp3,সাম্প্রতিক বছরগুলিতে নগরীর জনসংখ্যা হ্রাস পাচ্...
1,0177f2495e60.mp3,তোমার মনে হয় ও এখনও এটা করতে চায়?
2,01cbf95d03e2.mp3,এছাড়াও বিমানবন্দরটি ব্যাংকক আন্তর্জাতিক বিমান...
3,021dd2bbecb9.mp3,অবশেষে সেই সংগীত এক ভয়াবহ আতঙ্কের রূপ ধারণ করত।
4,02638823fd04.mp3,এসব ক্ষেত্রে ব্যক্তিগত তথ্য গোপন রাখা খুব কষ্টকর।
...,...,...
291,f8e67328f00f.mp3,পাশাপাশি এগুলো কখনও আদালতে প্রমাণিত হয়নি।
292,fd4f3d27f395.mp3,উইলিয়ামস কিশোরী বয়সে টেলিভিশনে অতিথি চরিত্রে...
293,fe2e15db209a.mp3,চ্যানেলটির পরিচালক রাজেন্দ্র শাক্য।
294,feb608a34c8a.mp3,মুক্তিযুদ্ধ শুরু হলে ওসমানী সীমান্ত পার হয়ে ভ...


In [8]:
dataset = load_dataset("audiofolder", data_dir="train_mp3_subsets/train_subset_1/")

Resolving data files: 100%|██████████| 297/297 [00:00<00:00, 236916.75it/s]
Extracting data files: 0it [00:00, ?it/s]
Generating train split: 296 examples [00:00, 3197.88 examples/s]


In [ ]:
# dataset["train"]=dataset["train"].filter(lambda x,y: x>y, input_columns=["up_votes","down_votes"])
# dataset["validation"]=dataset["validation"].filter(lambda x,y: x>y, input_columns=["up_votes","down_votes"])


In [ ]:
# import re
# chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\—\‘\'\‚\“\”\…]'

# def remove_special_characters(batch):
#     batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]) + " "
#     return batch

# dataset["train"]=dataset["train"].map(remove_special_characters)
# dataset["validation"]=dataset["validation"].map(remove_special_characters)



# dataset["train"]=dataset["train"].remove_columns(['up_votes','down_votes','age','gender','accent','locale','segment'])
# dataset["validation"]=dataset["validation"].remove_columns(['up_votes','down_votes','age','gender','accent','locale','segment'])


In [11]:
# !pip install bnunicodenormalizer

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for bnunicodenormalizer: filename=bnunicodenormalizer-0.1.6-py3-none-any.whl size=22841 sha256=ea984417dabd7675803fd86a907ce41349b17ba65d6736f471f153a314f58fbe
  Stored in directory: c:\users\moses\appdata\local\pip\cache\wheels\9e\ea\56\34bd9fbd2e2b0a0166461fb78518deebe2d6f8e8e028a2be7c
Successfully built bnunicodenormalizer


DEPRECATION: Loading egg at c:\users\moses\anaconda3\envs\ml_audio\lib\site-packages\sacremoses-0.0.43-py3.8.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [9]:
from bnunicodenormalizer import Normalizer 
bnorm=Normalizer()

def normalize(batch):
  
    _words = [bnorm(word)['normalized']  for word in batch["transcription"].split()]
    batch["transcription"] =  " ".join([word for word in _words if word is not None])
    return batch

dataset = dataset.map(normalize)

Map:   0%|          | 0/296 [00:00<?, ? examples/s]

Map: 100%|██████████| 296/296 [00:01<00:00, 213.73 examples/s]


In [10]:

from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("arijitx/wav2vec2-xls-r-300m-bengali")

c:\Users\Moses\anaconda3\envs\ml_audio\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Moses\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [11]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16_000))

In [12]:
import numpy as np

def prepare_dataset(batch):

    batch["audio"]["array"] = np.trim_zeros(batch["audio"]["array"], 'fb')
    audio = batch["audio"]
    

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=16000).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

In [13]:
# dataset_train = dataset['train'].map(prepare_dataset, remove_columns=dataset.column_names)
# dataset_validation = dataset.map(prepare_dataset, remove_columns=dataset.column_names)

ValueError: Column to remove ['train'] not in the dataset. Current columns in the dataset: ['audio', 'transcription']

In [14]:
def trim_silence(batch):
    arr = batch['input_values']
    
    try:
        _max = max(max(arr), -min(arr))
        old_length = len(arr)
        
        threshold = 30

        for i,e in enumerate(arr):
            if threshold*e>_max:
                break

        for j,e in enumerate(reversed(arr)):
            if threshold*e>_max:
                break

        batch['input_values'] = arr[i:old_length-j]
        batch['input_length'] = old_length -i -j
    except:
        print(batch['input_length'])
    return batch
    



In [17]:
dataset_train = dataset.map(trim_silence)

Map:   0%|          | 0/296 [00:00<?, ? examples/s]


KeyError: 'input_values'

In [ ]:
# dataset_train.push_to_hub("commonVoice_bn_train_preprocessed_trimmed")
# dataset_validation.push_to_hub("commonVoice_bn_validation_preprocessed")